In [1]:
!gdown --id 1Hrzb8-td5D0P4GLks3QBtzkCUzpszzP3

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Hrzb8-td5D0P4GLks3QBtzkCUzpszzP3
To: /content/TravelInsurancePrediction.csv
100% 115k/115k [00:00<00:00, 87.1MB/s]


In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 9.2 MB/s eta 0:00:00


# Import Lib

In [3]:
#Data manipulation

import pandas as pd
import numpy as np

#Data visualization

import matplotlib.pyplot as plt
import seaborn as sns

#Warning

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

#Preparation
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [ ]:
!pip install xgboost

In [ ]:
!pip install sklearn

In [ ]:
df = pd.read_csv('TravelInsurancePrediction.csv')
df.head()

In [ ]:
df['TravelInsurance'].value_counts()

In [ ]:
df = df.drop_duplicates(subset = ['Age','Employment Type','GraduateOrNot','AnnualIncome','FamilyMembers','ChronicDiseases','FrequentFlyer','EverTravelledAbroad',"TravelInsurance"])
df.shape

df['Employment Type']=df['Employment Type'].map({'Private Sector/Self Employed':1,'Government Sector':0})
df['GraduateOrNot']=df['GraduateOrNot'].map({'Yes':1,'No':0})
df['FrequentFlyer']=df['FrequentFlyer'].map({'No':0,'Yes':1})
df['EverTravelledAbroad']=df['EverTravelledAbroad'].map({'No':0,'Yes':1})
df.head()

In [ ]:
ft = [ 'Age', 'Employment Type', 'GraduateOrNot', 'AnnualIncome','FamilyMembers', 'ChronicDiseases', 'FrequentFlyer', 'EverTravelledAbroad']
x = df[ft]
y = df.TravelInsurance

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(x, y, test_size=0.2, random_state=42) #split data

In [ ]:
print("train_x size : ",train_x.size)
print("test_x size  : ",test_x.size)
print("train_y size : ",train_y.size)
print("test_y size  : ",test_y.size)

In [ ]:
test_x.shape

In [ ]:
test_y.value_counts()

In [ ]:
pd.DataFrame(train_y)

In [ ]:
print(pd.Series(train_y).value_counts())

# Oversampling Data Split dengan SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over, y_train_over = SMOTE().fit_resample(train_x, train_y)

In [ ]:
# Sebelum Oversampling
print('Target sebelum oversampling:')
print(pd.Series(train_y).value_counts())

In [ ]:
# Setelah Oversampling
print('Target setelah oversampling:')
print(pd.Series(y_train_over).value_counts())

In [ ]:
y_train_over

In [ ]:
X_train_over.info()

# DATA MODELLING

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV, cross_val_score
from imblearn.over_sampling import SMOTE

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.under_sampling import NearMiss
#from catboost import CatBoostClassifier

from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, precision_recall_curve, PrecisionRecallDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
#from yellowbrick.classifier import DiscriminationThreshold
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

In [ ]:
#model evaluation classification test (default)
def eval_classification(model):
    y_pred = model.predict(test_x)
    y_pred_train = model.predict(train_x)
    y_pred_proba = model.predict_proba(test_x)
    y_pred_proba_train = model.predict_proba(train_x)

    print("Accuracy (Test Set): %.2f" % accuracy_score(test_y, y_pred))
    print("Accuracy (Train Set): %.2f" % accuracy_score(train_y, y_pred_train))

    print("Precision (Test Set): %.2f" % precision_score(test_y, y_pred))
    print("Precision (Train Set): %.2f" % precision_score(train_y, y_pred_train))

    print("Recall (Test Set): %.2f" % recall_score(test_y, y_pred))
    print("Recall (Train Set): %.2f" % recall_score(train_y, y_pred_train))

    print("F1-Score (Test Set): %.2f" % f1_score(test_y, y_pred))
    print("F1-Score (Train Set): %.2f" % f1_score(train_y, y_pred_train))

    print("roc_auc (test-proba): %.2f" % roc_auc_score(test_y, y_pred_proba[:, 1]))
    print("roc_auc (train-proba): %.2f" % roc_auc_score(train_y, y_pred_proba_train[:, 1]))

    score = cross_validate(model, x, y, cv=5, scoring='roc_auc', return_train_score=True)
    print('roc_auc (crossval train): '+ str(score['train_score'].mean()))
    print('roc_auc (crossval test): '+ str(score['test_score'].mean()))


def cnf_test(model):# Membuat Confusion Matrix (Test)
    y_pred = model.predict(test_x)
    y_pred_train = model.predict(train_x)
    y_pred_proba = model.predict_proba(test_x)
    y_pred_proba_train = model.predict_proba(train_x)
    cnf_matrix = confusion_matrix(test_y, y_pred)
    print(classification_report(test_y, y_pred))
    sns.heatmap(cnf_matrix,cmap='coolwarm_r',annot=True,linewidth=0.5,fmt='d')
    plt.title('Confusion Matrix (Test)')
    plt.xlabel('Prediksi')
    plt.ylabel('Aktual')

def cnf_train(model):# Membuat Confusion Matrix (Test)
    y_pred = model.predict(test_x)
    y_pred_train = model.predict(train_x)
    y_pred_proba = model.predict_proba(test_x)
    y_pred_proba_train = model.predict_proba(train_x)
    cnf_matrix = confusion_matrix(train_y, y_pred_train)
    print(classification_report(train_y, y_pred_train))
    sns.heatmap(cnf_matrix,cmap='coolwarm_r',annot=True,linewidth=0.5,fmt='d')
    plt.title('Confusion Matrix (Train)')
    plt.xlabel('Prediksi')
    plt.ylabel('Aktual')

def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=x.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter(model):
    print(model.best_estimator_.get_params())

# ML Supervised Model

In [ ]:
# Declaring Functions
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate

def eval_classification1(model):
    y_pred = model.predict(test_x)
    y_pred_train = model.predict(train_x)
    y_pred_proba = model.predict_proba(test_x)
    y_pred_proba_train = model.predict_proba(train_x)


    acc_test = accuracy_score(test_y, y_pred)
    precision_test = precision_score(test_y, y_pred)
    recall_test = recall_score(test_y, y_pred)
    f1_test = f1_score(test_y, y_pred)
    roc_auc_train = roc_auc_score(train_y, y_pred_proba_train[:, 1])
    roc_auc_test = roc_auc_score(test_y, y_pred_proba[:, 1])

#     print("Accuracy (Test Set): %.2f" % accuracy_score(y_test, y_pred))
#     print("Precision (Test Set): %.2f" % precision_score(y_test, y_pred))
#     print("Recall (Test Set): %.2f" % recall_score(y_test, y_pred))
#     print("F1-Score (Test Set): %.2f" % f1_score(y_test, y_pred))

#     print("roc_auc (train-proba): %.2f" % roc_auc_score(y_train, y_pred_proba_train[:, 1]))
#     print("roc_auc (test-proba): %.2f" % roc_auc_score(y_test, y_pred_proba[:, 1]))

    score = cross_validate(model, x, y, cv=5, scoring='roc_auc', return_train_score=True)
    roc_auc_crossval_train = round(score['train_score'].mean(), 4)
    roc_auc_crossval_test = round(score['test_score'].mean(), 4)
    # print('roc_auc (crossval train): '+ str(round(score['train_score'].mean(), 4)))
    # print('roc_auc (crossval test): '+ str(round(score['test_score'].mean(), 4)))

    eval_result = {
        'Test Accuracy': acc_test,
        'Test Precision': precision_test,
        'Test Recall': recall_test,
        'Test F1': f1_test,
        'Train ROC AUC': roc_auc_train,
        'Test ROC AUC': roc_auc_test,
        'Crossval Train ROC AUC': roc_auc_crossval_train,
        'Crossval Test ROC AUC': roc_auc_crossval_test
    }
    return eval_result

def show_feature_importance1(model):
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter1(model):
    print(model.best_estimator_.get_params())

In [ ]:
# Importing all classifier
from sklearn.linear_model import LogisticRegression # import Logistic Regression dari sklearn
#from sklearn.neighbors import KNeighborsClassifier # import kNN dari sklearn
from sklearn.tree import DecisionTreeClassifier # import Decision Tree dari sklearn
from sklearn.ensemble import RandomForestClassifier # import Random Forest dari sklearn
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost dari sklearn
from xgboost import XGBClassifier #import XGBoost darti xgboost

#Evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.metrics import make_scorer,accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,log_loss
from sklearn.metrics import confusion_matrix

In [ ]:
# Modelling for Selected Algorithms

# Selected Models
models1 = ['Logistic Regression', 'Decision Tree', 'AdaBoost Classifier', 'XGB Classifier', 'Random Forest Classifier']
algorithms1 = [LogisticRegression, DecisionTreeClassifier, AdaBoostClassifier, XGBClassifier, RandomForestClassifier]

# List of results:
acc_test_ni = []
precision_test_ni = []
recall_test_ni = []
f1_test_ni = []
roc_auc_train_ni = []
roc_auc_test_ni = []
roc_auc_crossval_train_ni = []
roc_auc_crossval_test_ni = []

In [ ]:
# Evaluating All Models Using Unhandled CLass Imbalance Data
i = 0
random_seed = [DecisionTreeClassifier, RandomForestClassifier, AdaBoostClassifier]

for model_type in algorithms1:
    # Fitting and using the algorithms
    if model_type in random_seed:
        model = model_type(random_state=42)
    model = model_type(random_state=42)
    model.fit(train_x, train_y)
    print(models1[i])
    result = eval_classification1(model)

    # Adding the results to the list
    acc_test_ni.append(result['Test Accuracy'])
    precision_test_ni.append(result['Test Precision'])
    recall_test_ni.append(result['Test Recall'])
    f1_test_ni.append(result['Test F1'])
    roc_auc_train_ni.append(result['Train ROC AUC'])
    roc_auc_test_ni.append(result['Test ROC AUC'])
    roc_auc_crossval_train_ni.append(result['Crossval Train ROC AUC'])
    roc_auc_crossval_test_ni.append(result['Crossval Test ROC AUC'])

    i += 1

In [ ]:
result_df = pd.DataFrame(list(zip(models1, acc_test_ni, precision_test_ni, recall_test_ni, f1_test_ni, roc_auc_train_ni, roc_auc_test_ni, roc_auc_crossval_train_ni, roc_auc_crossval_test_ni)), columns=['Algorithms', 'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1', 'Train ROC AUC', 'Test ROC AUC', 'Crossval Train ROC AUC', 'Crossval Test ROC AUC'])

## All Basic Algorithm Result

In [ ]:
result_df

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state = 42)#, class_weight='balanced')
logreg.fit(train_x, train_y)
eval_classification(logreg)

In [ ]:
cnf_train(logreg)

In [ ]:
cnf_test(logreg)

## Decision Tree

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_x, train_y)
eval_classification(dt)

In [ ]:
cnf_train(dt)

In [ ]:
cnf_test(dt)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(random_state=42)
ada.fit(train_x, train_y)
eval_classification(ada)

In [ ]:
cnf_train(ada)

In [ ]:
cnf_test(ada)

## XGBoost

In [ ]:
from xgboost import XGBClassifier

xg = XGBClassifier(random_state=42)
xg.fit(train_x, train_y)
eval_classification(xg)

In [ ]:
cnf_train(xg)

In [ ]:
cnf_test(xg)

## Random Forest Model

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(train_x, train_y)
eval_classification(rf)

In [ ]:
cnf_train(rf)

In [ ]:
cnf_test(rf)

# Model Using SMOTE

In [ ]:
# Modelling for Selected Algorithms

# Selected Models
models2 = ['Logistic Regression', 'Decision Tree', 'AdaBoost Classifier', 'XGB Classifier', 'Random Forest Classifier']
algorithms2 = [LogisticRegression, DecisionTreeClassifier, AdaBoostClassifier, XGBClassifier, RandomForestClassifier]

# List of results:
acc_test_ni = []
precision_test_ni = []
recall_test_ni = []
f1_test_ni = []
roc_auc_train_ni = []
roc_auc_test_ni = []
roc_auc_crossval_train_ni = []
roc_auc_crossval_test_ni = []

In [ ]:
# Evaluating All Models Using SMOTE
i = 0
random_seed = [DecisionTreeClassifier, RandomForestClassifier, AdaBoostClassifier]

for model_type in algorithms1:
    # Fitting and using the algorithms
    if model_type in random_seed:
        model = model_type(random_state=42)
    model = model_type(random_state=42)
    model.fit(X_train_over, y_train_over)
    print(models2[i])
    result = eval_classification1(model)

    # Adding the results to the list
    acc_test_ni.append(result['Test Accuracy'])
    precision_test_ni.append(result['Test Precision'])
    recall_test_ni.append(result['Test Recall'])
    f1_test_ni.append(result['Test F1'])
    roc_auc_train_ni.append(result['Train ROC AUC'])
    roc_auc_test_ni.append(result['Test ROC AUC'])
    roc_auc_crossval_train_ni.append(result['Crossval Train ROC AUC'])
    roc_auc_crossval_test_ni.append(result['Crossval Test ROC AUC'])

    i += 1

In [ ]:
result_df1 = pd.DataFrame(list(zip(models2, acc_test_ni, precision_test_ni, recall_test_ni, f1_test_ni, roc_auc_train_ni, roc_auc_test_ni, roc_auc_crossval_train_ni, roc_auc_crossval_test_ni)), columns=['Algorithms', 'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1', 'Train ROC AUC', 'Test ROC AUC', 'Crossval Train ROC AUC', 'Crossval Test ROC AUC'])

## All SMOTE Based Algorithm

In [ ]:
result_df1

## Logistic Regression Using SMOTE

In [ ]:
lrs = LogisticRegression(random_state = 42)#, class_weight='balanced')
lrs.fit(X_train_over, y_train_over)
eval_classification(lrs)

In [ ]:
cnf_train(lrs)

In [ ]:
cnf_test(lrs)

## Decision Tree Using SMOTE

In [ ]:
dts = DecisionTreeClassifier(random_state=42)
dts.fit(X_train_over, y_train_over)
eval_classification(dts)

In [ ]:
cnf_train(dts)

In [ ]:
cnf_test(dts)

## AdaBoost Using SMOTE

In [ ]:
adas = AdaBoostClassifier(random_state=42)
adas.fit(X_train_over, y_train_over)
eval_classification(adas)

In [ ]:
cnf_train(adas)

In [ ]:
cnf_test(adas)

## XGBoost Using SMOTE

In [ ]:
from xgboost import XGBClassifier

xgs = XGBClassifier(random_state=42)
xgs.fit(X_train_over, y_train_over)
eval_classification(xgs)

In [ ]:
cnf_train(xgs)

In [ ]:

cnf_test(xgs)

## Random Forest Using SMOTE

In [ ]:
rfs = RandomForestClassifier(random_state=42)
rfs.fit(X_train_over, y_train_over)
eval_classification(rfs)

In [ ]:
cnf_train(rfs)

In [ ]:
cnf_test(rfs)

# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, learning_curve
import numpy as np
import matplotlib.pyplot as plt

#Menjadikan ke dalam bentuk dictionary
hyperparameters = {
                    'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)],
                    'min_child_weight' : [int(x) for x in np.linspace(1, 20, num = 11)],
                    'gamma' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'tree_method' : ['auto', 'exact', 'approx', 'hist'],

                    'colsample_bytree' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'eta' : [float(x) for x in np.linspace(0, 1, num = 100)],

                    'lambda' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'alpha' : [float(x) for x in np.linspace(0, 1, num = 11)]
                    }

# Init
from xgboost import XGBClassifier
xgsa = XGBClassifier(random_state=42)
xg_tuned = RandomizedSearchCV(xgsa, hyperparameters, cv=5, random_state=42, scoring='f1')
xg_tuned.fit(X_train_over, y_train_over)

# Predict & Evaluation
eval_classification(xg_tuned)

In [ ]:
cnf_train(xg_tuned)

In [ ]:
cnf_test(xg_tuned)

In [ ]:
# Membuat Learning Curve
train_sizes, train_scores, validation_scores = learning_curve(xg_tuned.best_estimator_, X_train_over, y_train_over, cv=5)
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='training score')
plt.plot(train_sizes, np.mean(validation_scores, axis=1), label='validation score')
plt.legend(loc='best')
plt.show()

# Menyimpan hasil gambar
plt.savefig('learning_curve.png')

## XGBoost+SMOTE

In [ ]:
#show_best_hyperparameter(xgs)
xgs_param_grid = {'n_estimators': np.linspace(100, 1000, 10, dtype=int),
                  'learning_rate': np.arange(0.05, 1, 0.05),
                  'max_depth': np.arange(3, 11, 1, dtype=int),
                  'min_child_weight': np.arange(1, 8, 1, dtype=int),
                  'tree_method' : ['auto', 'exact', 'approx', 'hist'],
                  'gamma' : np.arange(0,10,0.5),
                  #'lambda' : [float(x) for x in np.linspace(0, 1, num = 11)],
                  'alpha' : [float(x) for x in np.linspace(0, 1, num = 11)],
                  'colsample_bytree' : [float(x) for x in np.linspace(0, 1, num = 11)]}

In [ ]:
xgs_rs = RandomizedSearchCV(xgs, xgs_param_grid, scoring='f1', cv=5, random_state=42)

In [ ]:
xgs_rs.fit(X_train_over, y_train_over)

print(f'Best parameters: {xgs_rs.best_params_}')

In [ ]:
xgs2 = XGBClassifier(random_state=42,
                   tree_method= 'approx',
                   n_estimators= 100,
                   min_child_weight= 7,
                   max_depth= 4,
                   learning_rate= 0.8,
                   #lambda= 0.30000000000000004,
                   gamma= 7,
                   colsample_bytree= 1,
                   alpha= 0.9)

xgs2.fit(X_train_over, y_train_over)

# Predict & Evaluation
eval_classification(xgs2)

In [ ]:
cnf_train(xgs2)

In [ ]:
cnf_test(xgs2)

# Feature Importance

In [ ]:
show_feature_importance(xg)

5 Feature importance tertinggi adalah AnnualIncome, Family Members, Age, Ever Travelled Abroad, Chronic Diseases.

In [ ]:
import shap
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(test_x)
shap.summary_plot(shap_values[1], test_x)

# Business Insight

Dari hasil analisa, perusahaan bisa fokus pada pelanggan yang memiliki karakteristik gabungan dari 5 feature importance tertinggi (Age, annual income, ever travelled abroad, Family members, Chronic diseases) agar bisa menarik pelanggan baru yang tepat sasaran.

Dalam hal prospecting, perusahaan bisa memfokuskan penjualan ke calon klien yang sudah berkeluarga serta cenderung memiliki penyakit kronis yang sesuai dengan paket perlindungan asuransi dan pernah berpergian misalnya dengan memberikan promo terkait tiket perjalanan yang lebih murah apabila calon klien membeli paket asuransi travel. Hal tersebut dapat menarik minat pelanggan.